In [ ]:
!pip install tensorflow==2.15.0
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully unin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import glob
import numpy as np
import cv2

def load_train_lfw_data(data_dir):
    # Danh sách tất cả các tên thư mục (mỗi thư mục chứa các ảnh của một người)
    person_dirs = glob.glob(os.path.join(data_dir, '*'))

    images = []
    names = []

    for person_dir in person_dirs:
        person_name = os.path.basename(person_dir)  # Tên người
        image_paths = glob.glob(os.path.join(person_dir, '*.jpg'))

        for img_path in image_paths:
            img = cv2.imread(img_path)  # Đọc ảnh với 3 kênh màu (RGB)
            img = cv2.resize(img, (224, 224))  # Resize về kích thước mong muốn
            images.append(img)
            names.append(person_name)

    images = np.array(images)
    names = np.array(names)

    return images, names

data_dir = '/content/drive/MyDrive/Colab Notebooks/Train_ver3'
X, names = load_train_lfw_data(data_dir)

In [ ]:
# Encode nhãn thành số nguyên
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
names_encoded = le.fit_transform(names)

# Lưu encoder để dùng sau
import joblib
joblib.dump(le, '/content/drive/MyDrive/label_encoder_ver3.pkl')

['/content/drive/MyDrive/label_encoder_ver3.pkl']

In [ ]:
# Chia train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, names_encoded, test_size=0.2, stratify=names_encoded)

# Thông tin dữ liệu
print(X_train.shape, X_test.shape)

(772, 224, 224, 3) (194, 224, 224, 3)


In [ ]:
print(set(y_train))  # Xem có gì bất thường không
print(np.isnan(X_train).any())  # Kiểm tra NaN

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79}
False


In [ ]:
# Tạo mô hình backbone
import tensorflow as tf
from tensorflow.keras.layers import Dense, Lambda, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16

def _base_network():
    base_model = VGG16(include_top=False, weights='imagenet', input_tensor=Input(shape=(224, 224, 3)))

    #Mở 4 lớp cuối
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    for layer in base_model.layers[-4:]:
        layer.trainable = True

    x = Flatten()(base_model.output)
    x = Dense(128)(x)
    norm2 = Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)
    model = Model(inputs=base_model.input, outputs=norm2)
    return model

# Khởi tạo mô hình
model = _base_network()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
# Dùng Triplet Loss
import tensorflow_addons as tfa
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),
    loss=tfa.losses.TripletSemiHardLoss()
)

In [ ]:
gen_train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(62).repeat()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Tạo callback để lưu mô hình sau mỗi epoch
checkpoint_callback = ModelCheckpoint(
    '/content/drive/MyDrive/facenet_ver3.h5',
    monitor='loss',
    save_best_only=True,
    save_weights_only=False,  # Lưu toàn bộ mô hình (weights và architecture)
    mode='min',
    verbose=1  # In ra thông tin khi lưu checkpoint
)

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

print("TensorFlow version:", tf.__version__)
print("TensorFlow Addons version:", tfa.__version__)


TensorFlow version: 2.15.0
TensorFlow Addons version: 0.23.0


In [ ]:
try:
    import tensorflow as tf
    print("TensorFlow:", tf.__version__)
except ImportError:
    print("TensorFlow chưa được cài đặt hoặc import lỗi!")

try:
    import tensorflow_addons as tfa
    print("TensorFlow Addons:", tfa.__version__)
except ImportError:
    print("TensorFlow Addons chưa được cài đặt hoặc import lỗi!")

try:
    import numpy as np
    print("NumPy:", np.__version__)
except ImportError:
    print("NumPy chưa được cài đặt hoặc import lỗi!")

try:
    import cv2
    print("OpenCV:", cv2.__version__)
except ImportError:
    print("OpenCV chưa được cài đặt hoặc import lỗi!")

try:
    import sklearn
    print("scikit-learn:", sklearn.__version__)
except ImportError:
    print("scikit-learn chưa được cài đặt hoặc import lỗi!")

try:
    import joblib
    print("joblib:", joblib.__version__)
except ImportError:
    print("joblib chưa được cài đặt hoặc import lỗi!")


TensorFlow: 2.15.0
TensorFlow Addons: 0.23.0
NumPy: 1.23.5
OpenCV: 4.11.0
scikit-learn: 1.6.1
joblib: 1.5.0


In [ ]:
history = model.fit(
    gen_train,
    steps_per_epoch=30,
    epochs=15,
    callbacks=[checkpoint_callback]  # Thêm callback để checkpoint
)

Epoch 1/15
30/30 [==============================] - ETA: 0s - loss: 0.9574 
Epoch 1: loss improved from inf to 0.95743, saving model to /content/drive/MyDrive/facenet_ver3.h5


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


30/30 [==============================] - 1184s 39s/step - loss: 0.9574
Epoch 2/15
30/30 [==============================] - ETA: 0s - loss: 0.8651 
Epoch 2: loss improved from 0.95743 to 0.86505, saving model to /content/drive/MyDrive/facenet_ver3.h5
30/30 [==============================] - 1171s 39s/step - loss: 0.8651
Epoch 3/15
30/30 [==============================] - ETA: 0s - loss: 0.7405 
Epoch 3: loss improved from 0.86505 to 0.74052, saving model to /content/drive/MyDrive/facenet_ver3.h5
30/30 [==============================] - 1163s 39s/step - loss: 0.7405
Epoch 4/15
30/30 [==============================] - ETA: 0s - loss: 0.6157 
Epoch 4: loss improved from 0.74052 to 0.61573, saving model to /content/drive/MyDrive/facenet_ver3.h5
30/30 [==============================] - 1141s 39s/step - loss: 0.6157
Epoch 5/15
30/30 [==============================] - ETA: 0s - loss: 0.4963 
Epoch 5: loss improved from 0.61573 to 0.49630, saving model to /content/drive/MyDrive/facenet_ver3.h5


In [ ]:
history = model.fit(
    gen_train,
    steps_per_epoch=30,
    epochs=10,
    callbacks=[checkpoint_callback]  # Thêm callback để checkpoint
)

Epoch 1/10
30/30 [==============================] - ETA: 0s - loss: 0.0276 
Epoch 1: loss improved from 0.03791 to 0.02759, saving model to /content/drive/MyDrive/facenet_ver3.h5
30/30 [==============================] - 1151s 38s/step - loss: 0.0276
Epoch 2/10
30/30 [==============================] - ETA: 0s - loss: 0.0201 
Epoch 2: loss improved from 0.02759 to 0.02006, saving model to /content/drive/MyDrive/facenet_ver3.h5
30/30 [==============================] - 1153s 38s/step - loss: 0.0201
Epoch 3/10
30/30 [==============================] - ETA: 0s - loss: 0.0083 
Epoch 3: loss improved from 0.02006 to 0.00825, saving model to /content/drive/MyDrive/facenet_ver3.h5
30/30 [==============================] - 1169s 39s/step - loss: 0.0083
Epoch 4/10
30/30 [==============================] - ETA: 0s - loss: 0.0080 
Epoch 4: loss improved from 0.00825 to 0.00795, saving model to /content/drive/MyDrive/facenet_ver3.h5
30/30 [==============================] - 1152s 39s/step - loss: 0.0080


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

print("TensorFlow version:", tf.__version__)
print("TensorFlow Addons version:", tfa.__version__)

TensorFlow version: 2.15.0
TensorFlow Addons version: 0.23.0


In [ ]:
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa

model = load_model(
    '/content/drive/MyDrive/facenet_ver3.h5',
    custom_objects={'TripletSemiHardLoss': tfa.losses.TripletSemiHardLoss}
)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),
    loss=tfa.losses.TripletSemiHardLoss()
)

In [ ]:
gen_train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(62).repeat()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Tạo callback để lưu mô hình sau mỗi epoch
checkpoint_callback = ModelCheckpoint(
    '/content/drive/MyDrive/facenet_ver3.h5',
    monitor='loss',
    save_best_only=True,
    save_weights_only=False,  # Lưu toàn bộ mô hình (weights và architecture)
    mode='min',
    verbose=1  # In ra thông tin khi lưu checkpoint
)

In [ ]:
history = model.fit(
    gen_train,
    steps_per_epoch=30,
    epochs=20,
    callbacks=[checkpoint_callback]  # Thêm callback để checkpoint
)

Epoch 1/20
30/30 [==============================] - ETA: 0s - loss: 0.1604 
Epoch 1: loss improved from inf to 0.16040, saving model to /content/drive/MyDrive/facenet_ver3.h5


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


30/30 [==============================] - 1401s 47s/step - loss: 0.1604
Epoch 2/20
30/30 [==============================] - ETA: 0s - loss: 0.1046 
Epoch 2: loss improved from 0.16040 to 0.10457, saving model to /content/drive/MyDrive/facenet_ver3.h5
30/30 [==============================] - 1393s 46s/step - loss: 0.1046
Epoch 3/20
30/30 [==============================] - ETA: 0s - loss: 0.0612 
Epoch 3: loss improved from 0.10457 to 0.06119, saving model to /content/drive/MyDrive/facenet_ver3.h5
30/30 [==============================] - 1387s 46s/step - loss: 0.0612
Epoch 4/20
30/30 [==============================] - ETA: 0s - loss: 0.0299 
Epoch 4: loss improved from 0.06119 to 0.02992, saving model to /content/drive/MyDrive/facenet_ver3.h5
30/30 [==============================] - 1359s 46s/step - loss: 0.0299
Epoch 5/20
30/30 [==============================] - ETA: 0s - loss: 0.0273 
Epoch 5: loss improved from 0.02992 to 0.02727, saving model to /content/drive/MyDrive/facenet_ver3.h5


KeyboardInterrupt: 

#Test trên tập huấn luyện

In [ ]:
# Trích xuất embedding cho tập train và test
X_train_vecs = model.predict(X_train)
X_test_vecs = model.predict(X_test)

7/7 [==============================] - 119s 16s/step


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def _most_similarity(train_vecs, test_vec, train_labels):
    sims = cosine_similarity(train_vecs, test_vec)  # shape: (n_train, 1)
    idx = np.argmax(sims)
    return train_labels[idx]

In [ ]:
from sklearn.metrics import accuracy_score

y_preds = []
for vec in X_test_vecs:
    vec = vec.reshape(1, -1)
    y_pred = _most_similarity(X_train_vecs, vec, y_train)
    y_preds.append(y_pred)

print("Accuracy:", accuracy_score(y_test, y_preds))

Accuracy: 0.9587628865979382


#Test trên LFW

In [ ]:
import pandas as pd
import os

img_dir = "/content/drive/MyDrive/Colab Notebooks/lfw/lfw-deepfunneled/lfw-deepfunneled"
pairs_file = "/content/drive/MyDrive/Colab Notebooks/lfw/pairs.csv"

df = pd.read_csv(pairs_file, usecols=['name', 'imagenum1', 'imagenum2', 'Unnamed: 3'])

positive_pairs = df.iloc[:300]
negative_pairs = df.iloc[300:600]

pairs = []

# Positive pairs
for _, row in positive_pairs.iterrows():
    name = row['name']
    img1 = f"{img_dir}/{name}/{name}_{int(row['imagenum1']):04d}.jpg"
    img2 = f"{img_dir}/{name}/{name}_{int(row['imagenum2']):04d}.jpg"
    pairs.append((img1, img2, 1))

# Negative pairs
for _, row in negative_pairs.iterrows():
    name1 = row['name']
    img1 = f"{img_dir}/{name1}/{name1}_{int(row['imagenum1']):04d}.jpg"
    name2 = row['imagenum2']
    img2 = f"{img_dir}/{name2}/{name2}_{int(row['Unnamed: 3']):04d}.jpg"
    pairs.append((img1, img2, 0))

In [ ]:
import cv2
def preprocess_image(img_path):
    img = cv2.imread(img_path)  # Mặc định là BGR
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Đổi sang RGB
    img = cv2.resize(img, (224, 224))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)  # shape: (1, 224, 224, 3)
    return img

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np

# Đánh giá mô hình trên 1 ngưỡng
def evaluate_model(model, pairs, threshold):
    correct = 0
    total = len(pairs)
    for img1_path, img2_path, label in tqdm(pairs, desc=f"Evaluating (threshold={threshold:.2f})", leave=False):
        try:
            img1 = preprocess_image(img1_path)
            img2 = preprocess_image(img2_path)

            vec1 = model.predict(img1)[0]
            vec2 = model.predict(img2)[0]

            sim = cosine_similarity([vec1], [vec2])[0][0]
            pred = 1 if sim >= threshold else 0

            if pred == label:
                correct += 1
        except Exception as e:
            print(f"Lỗi xử lý cặp ảnh {img1_path} và {img2_path}: {e}")
    return correct / total if total > 0 else 0

# Tìm threshold tốt nhất
def find_best_threshold(model, pairs, thresholds):
    best_threshold = 0
    best_acc = 0
    for threshold in thresholds:
        acc = evaluate_model(model, pairs, threshold)
        print(f"Threshold: {threshold:.2f} → Accuracy: {acc:.4f}")
        if acc > best_acc:
            best_acc = acc
            best_threshold = threshold
    return best_threshold, best_acc

# Danh sách ngưỡng cần kiểm tra
thresholds_to_try = [0.84, 0.86]

# Tìm threshold tối ưu
best_threshold, best_accuracy = find_best_threshold(model, pairs, thresholds_to_try)
print("\nBest threshold:", best_threshold)
print("Best accuracy :", best_accuracy)

Evaluating (threshold=0.84):   0%|          | 0/600 [00:00<?, ?it/s]

1/1 [==============================] - 1s 580ms/step


Evaluating (threshold=0.84):   0%|          | 1/600 [00:01<13:10,  1.32s/it]

1/1 [==============================] - 1s 543ms/step


Evaluating (threshold=0.84):   0%|          | 2/600 [00:02<12:39,  1.27s/it]

...

Evaluating (threshold=0.86): 100%|█████████▉| 598/600 [14:27<00:03,  1.59s/it]

1/1 [==============================] - 1s 849ms/step


Evaluating (threshold=0.86): 100%|█████████▉| 599/600 [14:29<00:01,  1.73s/it]

1/1 [==============================] - 1s 536ms/step


Threshold: 0.86 → Accuracy: 0.6883

Best threshold: 0.86
Best accuracy : 0.6883333333333334


#3000 cặp test

In [ ]:
import pandas as pd
import os

img_dir = "/content/drive/MyDrive/Colab Notebooks/lfw/lfw-deepfunneled/lfw-deepfunneled"
pairs_file = "/content/drive/MyDrive/Colab Notebooks/lfw/3000test.csv"

df = pd.read_csv(pairs_file, usecols=['name', 'imagenum1', 'imagenum2', 'Unnamed: 3'])

positive_pairs = df.iloc[:1500]
negative_pairs = df.iloc[1500:3000]

pairs = []

# Positive pairs
for _, row in positive_pairs.iterrows():
    name = row['name']
    img1 = f"{img_dir}/{name}/{name}_{int(row['imagenum1']):04d}.jpg"
    img2 = f"{img_dir}/{name}/{name}_{int(row['imagenum2']):04d}.jpg"
    pairs.append((img1, img2, 1))

# Negative pairs
for _, row in negative_pairs.iterrows():
    name1 = row['name']
    img1 = f"{img_dir}/{name1}/{name1}_{int(row['imagenum1']):04d}.jpg"
    name2 = row['imagenum2']
    img2 = f"{img_dir}/{name2}/{name2}_{int(row['Unnamed: 3']):04d}.jpg"
    pairs.append((img1, img2, 0))

In [ ]:
import cv2
def preprocess_image(img_path):
    img = cv2.imread(img_path)  # Mặc định là BGR
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Đổi sang RGB
    img = cv2.resize(img, (224, 224))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)  # shape: (1, 224, 224, 3)
    return img

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np

# Đánh giá mô hình trên 1 ngưỡng
def evaluate_model(model, pairs, threshold):
    correct = 0
    total = len(pairs)
    for img1_path, img2_path, label in tqdm(pairs, desc=f"Evaluating (threshold={threshold:.2f})", leave=False):
        try:
            img1 = preprocess_image(img1_path)
            img2 = preprocess_image(img2_path)

            vec1 = model.predict(img1)[0]
            vec2 = model.predict(img2)[0]

            sim = cosine_similarity([vec1], [vec2])[0][0]
            pred = 1 if sim >= threshold else 0

            if pred == label:
                correct += 1
        except Exception as e:
            print(f"Lỗi xử lý cặp ảnh {img1_path} và {img2_path}: {e}")
    return correct / total if total > 0 else 0

# Tìm threshold tốt nhất
def find_best_threshold(model, pairs, thresholds):
    best_threshold = 0
    best_acc = 0
    for threshold in thresholds:
        acc = evaluate_model(model, pairs, threshold)
        print(f"Threshold: {threshold:.2f} → Accuracy: {acc:.4f}")
        if acc > best_acc:
            best_acc = acc
            best_threshold = threshold
    return best_threshold, best_acc

# Danh sách ngưỡng cần kiểm tra
thresholds_to_try = [0.86]

# Tìm threshold tối ưu
best_threshold, best_accuracy = find_best_threshold(model, pairs, thresholds_to_try)

Evaluating (threshold=0.86):   0%|          | 0/3000 [00:00<?, ?it/s]

1/1 [==============================] - 1s 607ms/step


Evaluating (threshold=0.86):   0%|          | 1/3000 [00:01<1:04:30,  1.29s/it]

1/1 [==============================] - 1s 919ms/step


Evaluating (threshold=0.86):   0%|          | 2/3000 [00:03<1:25:57,  1.72s/it]

...

Evaluating (threshold=0.86):  84%|████████▍ | 2519/3000 [1:28:16<18:28,  2.30s/it]

1/1 [==============================] - 1s 573ms/step


Evaluating (threshold=0.86):  84%|████████▍ | 2520/3000 [1:28:18<17:07,  2.14s/it]

1/1 [==============================] - 1s 727ms/step


Evaluating (threshold=0.86):  84%|████████▍ | 2521/3000 [1:28:19<15:28,  1.94s/it]

#Chạy lại 3000 test

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa

model = load_model(
    '/content/drive/MyDrive/facenet_ver3.h5',
    custom_objects={'TripletSemiHardLoss': tfa.losses.TripletSemiHardLoss}
)

In [ ]:
import pandas as pd
import os

img_dir = "/content/drive/MyDrive/Colab Notebooks/lfw/lfw-deepfunneled/lfw-deepfunneled"
pairs_file = "/content/drive/MyDrive/Colab Notebooks/lfw/3000test.csv"

df = pd.read_csv(pairs_file, usecols=['name', 'imagenum1', 'imagenum2', 'Unnamed: 3'])

positive_pairs = df.iloc[:1500]
negative_pairs = df.iloc[1500:3000]

pairs = []

# Positive pairs
for _, row in positive_pairs.iterrows():
    name = row['name']
    img1 = f"{img_dir}/{name}/{name}_{int(row['imagenum1']):04d}.jpg"
    img2 = f"{img_dir}/{name}/{name}_{int(row['imagenum2']):04d}.jpg"
    pairs.append((img1, img2, 1))

# Negative pairs
for _, row in negative_pairs.iterrows():
    name1 = row['name']
    img1 = f"{img_dir}/{name1}/{name1}_{int(row['imagenum1']):04d}.jpg"
    name2 = row['imagenum2']
    img2 = f"{img_dir}/{name2}/{name2}_{int(row['Unnamed: 3']):04d}.jpg"
    pairs.append((img1, img2, 0))

In [ ]:
import cv2
def preprocess_image(img_path):
    img = cv2.imread(img_path)  # Mặc định là BGR
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Đổi sang RGB
    img = cv2.resize(img, (224, 224))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)  # shape: (1, 224, 224, 3)
    return img

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np

# Đánh giá mô hình trên 1 ngưỡng
def evaluate_model(model, pairs, threshold):
    correct = 0
    total = len(pairs)
    for img1_path, img2_path, label in tqdm(pairs, desc=f"Evaluating (threshold={threshold:.2f})", leave=False):
        try:
            img1 = preprocess_image(img1_path)
            img2 = preprocess_image(img2_path)

            vec1 = model.predict(img1)[0]
            vec2 = model.predict(img2)[0]

            sim = cosine_similarity([vec1], [vec2])[0][0]
            pred = 1 if sim >= threshold else 0

            if pred == label:
                correct += 1
        except Exception as e:
            print(f"Lỗi xử lý cặp ảnh {img1_path} và {img2_path}: {e}")
    return correct / total if total > 0 else 0

# Tìm threshold tốt nhất
def find_best_threshold(model, pairs, thresholds):
    best_threshold = 0
    best_acc = 0
    for threshold in thresholds:
        acc = evaluate_model(model, pairs, threshold)
        print(f"Threshold: {threshold:.2f} → Accuracy: {acc:.4f}")
        if acc > best_acc:
            best_acc = acc
            best_threshold = threshold
    return best_threshold, best_acc

# Danh sách ngưỡng cần kiểm tra
thresholds_to_try = [0.87]

# Tìm threshold tối ưu
best_threshold, best_accuracy = find_best_threshold(model, pairs, thresholds_to_try)

Evaluating (threshold=0.87):   0%|          | 0/3000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 317ms/step


Evaluating (threshold=0.87):   0%|          | 1/3000 [00:00<35:36,  1.40it/s]

1/1 [==============================] - 0s 293ms/step


Evaluating (threshold=0.87):   0%|          | 2/3000 [00:01<34:41,  1.44it/s]

...

Evaluating (threshold=0.87): 100%|█████████▉| 2998/3000 [41:27<00:01,  1.24it/s]

1/1 [==============================] - 0s 323ms/step


Evaluating (threshold=0.87): 100%|█████████▉| 2999/3000 [41:28<00:00,  1.24it/s]

1/1 [==============================] - 0s 301ms/step


Threshold: 0.87 → Accuracy: 0.6643
